# Clean data scrapped from Monash handbook

The cleaned data will be the input to the course planner for Monash University Malaysia undergraduate computer science students.

In [44]:
import pandas as pd
import re
import os
import sys
sys.path.append('..')
from config import data_dir, chrome_driver_dir

The raw data contains details on all FIT units (and two MAT units) offered by Monash University.

In [45]:
unit = pd.read_json(os.path.join(data_dir, 'fit_unit.json'))
unit.head()

,code,name,offers,rule,credit_points
0,FIT2032,Industry-based learning,"[{'location': 'Clayton', 'period': 'First seme...",Prerequisite:\nAvailable to students in all un...,18
1,FIT1046,Interactive media foundations,"[{'location': 'Clayton', 'period': 'Second sem...",,6
2,FIT1045,Algorithms and programming fundamentals in python,"[{'location': 'Malaysia', 'period': 'October i...",Prerequisite: VCE Mathematics Methods or Speci...,6
3,FIT1033,Foundations of 3D,"[{'location': 'Clayton', 'period': 'First seme...","Prohibition: FIT2015, DIS1911",6
4,FIT1047,"Introduction to computer systems, networks and...","[{'location': 'Malaysia', 'period': 'October i...",Prohibitions: FIT1031 and FIT2078,6


## Unit offerings

Separate offering details from `unit` to another dataframe.

In [46]:
offers = []

for i in range(len(unit)):
    offer = pd.json_normalize(unit.offers[i])
    offer['code'] = unit.code[i]
    offers.append(offer)
    
offering = pd.concat(offers).reset_index(drop=True)
unit = unit.drop('offers', axis=1)
offering.head()

,location,period,mode,code
0,Clayton,First semester,On-campus,FIT2032
1,Clayton,Second semester,On-campus,FIT2032
2,Clayton,Second semester,On-campus,FIT1046
3,Malaysia,October intake,On-campus,FIT1045
4,Clayton,First semester,On-campus,FIT1045


In [47]:
offering['period'].unique()

array(['First semester', 'Second semester', 'October intake',
       'First semester (Fully flex)', 'Summer semester A',
       'Full year (Fully flex)', 'Summer semester B', 'Full year'],
      dtype=object)

Clean `offering.period`:
* Remove extra description
* Break down unit offerings labelled as 'Full year' into 'First sesmter' & 'Second semester'

In [48]:
offering.loc[offering['period'] == 'First semester (Fully flex)', 'period'] = 'First semester'
offering.loc[offering['period'] == 'Second semester (Fully flex)', 'period'] = 'Second semester'

s1 = offering[(offering['period'] == 'Full year')|(offering['period'] == 'Full year (Fully flex)')].copy()
s2 = s1.copy()
s1['period'] = 'First semester'
s2['period'] = 'Second semester'
offering = offering.append(s1)
offering = offering.append(s2)
offering = offering[offering['period'] != 'Full year']

offering.head()

,location,period,mode,code
0,Clayton,First semester,On-campus,FIT2032
1,Clayton,Second semester,On-campus,FIT2032
2,Clayton,Second semester,On-campus,FIT1046
3,Malaysia,October intake,On-campus,FIT1045
4,Clayton,First semester,On-campus,FIT1045


Since only the data for undergraduate units in Malaysia campus is required, filter `offering` by location, and remove rows that are honours units (FIT4xxx).

Some modifications are made:
* FIT3164 & FIT3163 (Data Science project 1 & 2) is added into unit offerings so that it is possible to complete a data science major in Malaysia campus.
* Remove FIT3045 & FIT3199 (Industry work experience) as this can be completed during summer semester.

In [49]:
ds_project = offering[((offering['code'] == 'FIT3164') | 
                       (offering['code'] == 'FIT3163')) & 
                      (offering['mode'] == 'On-campus')].copy()
ds_project['location'] = 'Malaysia'
offering = offering.append(ds_project)

offering_msia = offering[offering['location']=='Malaysia']
offering_msia = offering_msia[~offering_msia['code'].str.contains('^FIT4', regex=True)]
offering_msia = offering_msia[(offering_msia['code'] != 'FIT3045') & (offering_msia['code'] != 'FIT3199')]

offering_msia.head()

,location,period,mode,code
3,Malaysia,October intake,On-campus,FIT1045
5,Malaysia,First semester,On-campus,FIT1045
8,Malaysia,Second semester,On-campus,FIT1045
11,Malaysia,October intake,On-campus,FIT1047
13,Malaysia,First semester,On-campus,FIT1047


Save `offering_msia` (excluding `mode`) as csv file.

In [50]:
offering_msia.drop(
    ['mode', 'location'], axis=1).to_csv(os.path.join(data_dir, 'offering_msia.csv'), index=False)

## Unit details

Obtain the units that are offered in Malaysia campus for at least one teaching period.

In [51]:
unit_msia = unit.merge(offering_msia, on="code", how='inner')
unit_msia = unit_msia.groupby('code').first().reset_index().drop(['location', 'period', 'mode'], axis=1)
unit_msia = unit_msia.reset_index(drop=True)
unit_msia.head()

,code,name,rule,credit_points
0,FIT1008,Introduction to computer science,"Prerequisites: ((One of FIT1040, ECE2071, FIT1...",6
1,FIT1043,Introduction to data science,Prerequisite: Or VCE Mathematics Methods Or Sp...,6
2,FIT1045,Algorithms and programming fundamentals in python,Prerequisite: VCE Mathematics Methods or Speci...,6
3,FIT1047,"Introduction to computer systems, networks and...",Prohibitions: FIT1031 and FIT2078,6
4,FIT1051,Programming fundamentals in java,Prohibition: FIT1002,6


### Prerequisite units

Get the substrings containg prerequisite information.

In [52]:
prereq = unit_msia.rule.str.split('Prerequisite', expand=True)
# prereq = prereq[1] + prereq[2].fillna('')
# prereq = prereq.str.split('Prohibition', expand=True)[0]
prereq = prereq[1].str.split('Prohibition', expand=True)[0]
prereq.head()

0    s: ((One of FIT1040, ECE2071, FIT1002) and FIT...
1    : Or VCE Mathematics Methods Or Specialist Mat...
2    : VCE Mathematics Methods or Specialist Mathem...
3                                                 None
4                                                 None
Name: 0, dtype: object

Observe the that these prerequisite options are invalid because each of them involves a unit that is not offered in Malaysia, so they can be removed from the string.

In [53]:
tmp = prereq.str.findall('(or \(.*\))|(\(.*\) or)')
tmp = tmp[~pd.isna(tmp)]
tmp[tmp.map(len) > 0]

0     [(, ((One of FIT1040, ECE2071, FIT1002) and FI...
7                        [(or (FIT1040 and FIT1029), )]
12                       [(, (ENG1003 and ENG1060) or)]
20    [(or (FIT2001 and (FIT2004 or FIT2024 or FIT20...
Name: 0, dtype: object

In [54]:
prereq = prereq.replace(to_replace='(or \(.*\))|(\(.*\) or)', value='', regex=True)

Write a regex to identify groups of unit, i.e., at least one unit from each group must be completed to satisfy the prequiresite requirement.

In [55]:
code = "[A-Z]{3}[0-9]{4}"
regex = ["(ne of((,)|[A-Z]|[0-9]| |(or))+{})".format(code), 
         "(({}, )*{} \(?or {})".format(code, code, code),
         "({})".format(code)]
regex = '|'.join(regex)
regex

'(ne of((,)|[A-Z]|[0-9]| |(or))+[A-Z]{3}[0-9]{4})|(([A-Z]{3}[0-9]{4}, )*[A-Z]{3}[0-9]{4} \\(?or [A-Z]{3}[0-9]{4})|([A-Z]{3}[0-9]{4})'

Extract the unit code within the groups of unit. `prereq` now is a series of nested list, e.g., `[[FIT1047, FIT1031], [FIT1045, FIT1048, FIT1050]]` represents (one of FIT1047, FIT1031) and (one of FIT1045, FIT1048, FIT1050).

In [56]:
prereq = prereq.str.findall(regex)
prereq = prereq.fillna("")

for i in range(len(prereq)):
    match = prereq[i]
    prereq[i] = [None] * len(prereq[i])
    for j, item in enumerate(match):
        prereq[i][j] = re.findall(code, ''.join(item))

prereq.head()

0    [[FIT1045, FIT1053]]
1                      []
2                      []
3                      []
4                      []
Name: 0, dtype: object

### Required credit point & prohibition units

Identify the credit point required as part of the prerequisite, if this is not mentioned in the original string, the credit point required is 0.

In [57]:
req_point = unit_msia['rule'].str.findall('([0-9]+ (credit )?point)')
req_point = req_point.astype(str)
req_point = req_point.str.extract("([0-9]+)")
req_point = req_point.fillna('0')

Get the substrings containing describing the prohibitions, then extract the unit code from the substrings.

In [58]:
prohibition = unit_msia.rule.str.split('Prohibition', expand=True)[1]
prohibition = prohibition.str.split('requisite', expand=True)[0]
prohibition = prohibition.str.findall('[A-Z]{3}[0-9]{4}')
prohibition[pd.isna(prohibition)] = "[]"
prohibition.head()

0    [CSE1303, CSC1030, FIT1015, FIT2085]
1                                      []
2                      [FIT1029, FIT1053]
3                      [FIT1031, FIT2078]
4                               [FIT1002]
Name: 0, dtype: object

### Finalising the dataframe

Append the cleaned rule columns to `unit_msia` and drop the original rule column.

In [59]:
unit_msia['prerequisite'] = prereq
unit_msia['req_point'] = req_point
unit_msia['prohibition'] = prohibition
unit_msia = unit_msia.drop('rule', axis=1)

Save `unit_msia` as csv file.

In [60]:
unit_msia.to_csv(os.path.join(data_dir, 'fit_unit_msia.csv'), index=False)
unit_msia.head()

,code,name,credit_points,prerequisite,req_point,prohibition
0,FIT1008,Introduction to computer science,6,"[[FIT1045, FIT1053]]",0,"[CSE1303, CSC1030, FIT1015, FIT2085]"
1,FIT1043,Introduction to data science,6,[],0,[]
2,FIT1045,Algorithms and programming fundamentals in python,6,[],0,"[FIT1029, FIT1053]"
3,FIT1047,"Introduction to computer systems, networks and...",6,[],0,"[FIT1031, FIT2078]"
4,FIT1051,Programming fundamentals in java,6,[],0,[FIT1002]
